In [1]:
import pickle
import functions_reading as reading

In [22]:
def anotate_Cluster(clusters_d, data):
    level = data['level']
    clusters_members_d = data['merging_data']['jclu_d']
    for cluster_id in clusters_members_d:
        for netid in clusters_members_d[cluster_id]:
             clusters_d[netid][level] = cluster_id
    if level < 13:
        positive_clusters_id = data['children_clusters']
        for pclusterid in positive_clusters_id:
            clusters_d = anotate_Cluster(clusters_d, data['children_clusters'][pclusterid])
    return clusters_d

def write_String_Clusters(clusters_d, netid_pmid_d):
    """Have to pass the netid instead of the pmid because there are duplicated pmid"""
    out_file = ''
    for netid in clusters_d:
        n = clusters_d[netid]
        out_file += (str(netid_pmid_d[netid]) + '\t' + str(n[1]) + '\t' + str(n[2]) + '\t' + 
            str(n[3]) + '\t' + str(n[4]) + '\t' + str(n[5]) + '\t' + str(n[6]) + '\t' + str(n[7]) + '\t' + str(n[8]) +
            '\t' + str(n[9]) + '\t' + str(n[10]) + '\t' + str(n[11]) + '\t' + str(n[12]) + '\t' + str(n[13]) + '\n')
    while out_file[-1] == '\n':
        out_file = out_file[:-1]
    return out_file

def write_String_SR(year_sr_d):
    out_file = ''
    for year in year_sr_d:
        for sr in year_sr_d[year]:
            for pmid in year_sr_d[year][sr]:
                out_file +=  str(year) + '\t' + str(sr)+ '\t' + str(pmid) + '\n'
    while out_file[-1] == '\n':
        out_file = out_file[:-1]
    return out_file

def nedid_L(cs):
    netid_set = set([netid for pair in cs['parsed_network'] for netid in pair])
    netid_l = sorted(list(netid_set))
    return netid_l

In [5]:
f = open('cs_2014.pickle', 'rb')
cs_2014 = pickle.load(f)
f.close()
f = open('cs_2015.pickle', 'rb')
cs_2015 = pickle.load(f)
f.close()
f = open('cs_2016.pickle', 'rb')
cs_2016 = pickle.load(f)
f.close()
netid_pmid = reading.p_Tab_Delimited('netid_pmid.txt')
netid_pmid_d = {int(netid): int(pmid) for netid, pmid in netid_pmid}

In [6]:
netid_l_2014 = nedid_L(cs_2014)
netid_l_2015 = nedid_L(cs_2015)
netid_l_2016 = nedid_L(cs_2016)

In [7]:
clusters_d_2014 = {}
for netid in netid_l_2014:
    clusters_d_2014[netid] = {x:'-' for x in range(1,14)}
    
clusters_d_2015 = {}
for netid in netid_l_2015:
    clusters_d_2015[netid] = {x:'-' for x in range(1,14)}
    
clusters_d_2016 = {}
for netid in netid_l_2016:
    clusters_d_2016[netid] = {x:'-' for x in range(1,14)}

In [8]:
clusters_d_2014 = anotate_Cluster(clusters_d_2014, cs_2014['level_data'])
clusters_d_2015 = anotate_Cluster(clusters_d_2015, cs_2015['level_data'])
clusters_d_2016 = anotate_Cluster(clusters_d_2016, cs_2016['level_data'])

In [9]:
out_file_2014 = write_String_Clusters(clusters_d_2014, netid_pmid_d)
out_file_2015 = write_String_Clusters(clusters_d_2015, netid_pmid_d)
out_file_2016 = write_String_Clusters(clusters_d_2016, netid_pmid_d)

In [10]:
f = open('pmid_clusters_2014.txt', 'w')
f.write(out_file_2014)
f.close()

f = open('pmid_clusters_2015.txt', 'w')
f.write(out_file_2015)
f.close()

f = open('pmid_clusters_2016.txt', 'w')
f.write(out_file_2016)
f.close()

In [11]:
f = open('clean_ref.pickle', 'rb')
clean_ref = pickle.load(f)
f.close()

In [19]:
sr_positives = {}
sr_retrieved = {}
for year in clean_ref:
    sr_positives[year] = {}
    sr_retrieved[year] = {}
    for sr in clean_ref[year]:
        sr_positives[year][sr] = [netid_pmid_d[netid] for netid in clean_ref[year][sr]['positives_retrieved_in_net']]
        sr_retrieved[year][sr] = [netid_pmid_d[netid] for netid in clean_ref[year][sr]['retrieved_in_net']]

In [25]:
string_sr_positives = write_String_SR(sr_positives)
string_sr_retrieved = write_String_SR(sr_retrieved)

In [27]:
f = open('sr_positives.txt', 'w')
f.write(string_sr_positives)
f.close()

f = open('sr_retrieved.txt', 'w')
f.write(string_sr_retrieved)
f.close()